1. 导入必要的库

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import svd
from scipy.spatial.transform import Rotation
import time

2. DH参数变换矩阵计算

In [ ]:

def dh_transform(a, alpha, d, theta):
    """
    计算DH变换矩阵
    
    参数:
    a: 连杆长度
    alpha: 连杆扭角(弧度)
    d: 连杆偏距
    theta: 关节角度(弧度)
    
    返回:
    T: 4x4的DH变换矩阵
    """
    # 构建DH变换矩阵
    ct = np.cos(theta)
    st = np.sin(theta)
    ca = np.cos(alpha)
    sa = np.sin(alpha)
    
    T = np.array([
        [ct, -st, 0, a],
        [st*ca, ct*ca, -sa, -d*sa],
        [st*sa, ct*sa, ca, d*ca],
        [0, 0, 0, 1]
    ])
    
    return T

3. 前向运动学计算函数

In [ ]:
def forward_kinematics(angles, dh_params):
    """
    计算6轴机器人正向运动学
    
    参数:
        angles: 关节角度 [theta1, theta2, theta3, theta4, theta5, theta6]
        dh_params: DH参数字典
        
    返回:
        T: 末端执行器相对于基座的变换矩阵，4x4 np.array
    """
    theta1, theta2, theta3, theta4, theta5, theta6 = angles
    
    # 提取DH参数
    a0 = dh_params.get('a0', 0)
    a1 = dh_params.get('a1', 0)
    a2 = dh_params.get('a2', 0)
    a3 = dh_params.get('a3', 0)
    a4 = dh_params.get('a4', 0)
    a5 = dh_params.get('a5', 0)
    
    d1 = dh_params.get('d1', 0)
    d2 = dh_params.get('d2', 0)
    d3 = dh_params.get('d3', 0)
    d4 = dh_params.get('d4', 0)
    d5 = dh_params.get('d5', 0)
    d6 = dh_params.get('d6', 0)
    
    alpha0 = dh_params.get('alpha0', 0)
    alpha1 = dh_params.get('alpha1', 0)
    alpha2 = dh_params.get('alpha2', 0)
    alpha3 = dh_params.get('alpha3', 0)
    alpha4 = dh_params.get('alpha4', 0)
    alpha5 = dh_params.get('alpha5', 0)
    
    # 计算各个变换矩阵
    T0_1 = dh_transform(a0, alpha0, d1, theta1)
    T1_2 = dh_transform(a1, alpha1, d2, theta2-np.pi/2)  # 第二个关节偏移90度
    T2_3 = dh_transform(a2, alpha2, d3, theta3)
    T3_4 = dh_transform(a3, alpha3, d4, theta4)
    T4_5 = dh_transform(a4, alpha4, d5, theta5)
    T5_6 = dh_transform(a5, alpha5, d6, theta6+np.pi) #  第六个关节偏移180度
    
    # 计算总变换矩阵
    T0_2 = np.dot(T0_1, T1_2)
    T0_3 = np.dot(T0_2, T2_3)
    T0_4 = np.dot(T0_3, T3_4)
    T0_5 = np.dot(T0_4, T4_5)
    T0_6 = np.dot(T0_5, T5_6)
     
    return T0_6

def forward_kinematics_first3joints(angles, dh_params):
    """
    计算前3个关节的正向运动学，用于LM算法中的关节1-3求解
    
    参数:
        angles: 关节角度 [theta1, theta2, theta3]
        dh_params: DH参数字典
        
    返回:
        T: 第3个关节坐标系相对于基座的变换矩阵，4x4 np.array
    """
    theta1, theta2, theta3 = angles
    
    # 提取DH参数
    a0 = dh_params.get('a0', 0)
    a1 = dh_params.get('a1', 0)
    a2 = dh_params.get('a2', 0)
    
    d1 = dh_params.get('d1', 0)
    d2 = dh_params.get('d2', 0)
    d3 = dh_params.get('d3', 0)
    
    alpha0 = dh_params.get('alpha0', 0)
    alpha1 = dh_params.get('alpha1', 0)
    alpha2 = dh_params.get('alpha2', 0)
    
    # 计算各个变换矩阵
    T0_1 = dh_transform(a0, alpha0, d1, theta1)
    T1_2 = dh_transform(a1, alpha1, d2, theta2-np.pi/2)  # 第二个关节偏移90度
    T2_3 = dh_transform(a2, alpha2, d3, theta3)
    
    # 计算总变换矩阵
    T0_2 = np.dot(T0_1, T1_2)
    T0_3 = np.dot(T0_2, T2_3)
    

    return T0_3

4. 位姿误差计算函数

In [ ]:
def compute_pose_error(T_current, T_desired):
    """
    计算两个位姿之间的误差
    
    参数:
        T_current: 当前变换矩阵，4x4 np.array
        T_desired: 目标变换矩阵，4x4 np.array
    
    返回:
        error: 位姿误差向量，6x1 np.array (前3个元素是位置误差，后3个元素是姿态误差)
    """
    # 计算位置误差 (简单的欧几里得距离)
    pos_error = T_desired[:3, 3] - T_current[:3, 3]
    
    # 计算旋转误差 (使用轴角表示法)
    R_current = T_current[:3, :3]
    R_desired = T_desired[:3, :3]
    
    # 计算旋转误差矩阵 R_d * R_c^T
    R_error = R_desired @ R_current.T
    
    
    # 将旋转矩阵转换为轴角表示
    angle = np.arccos((np.trace(R_error) - 1) / 2)
    
    if abs(angle) < 1e-10:
        axis = np.zeros(3)
    else:
        axis =  np.array([
            R_error[2, 1] - R_error[1, 2],
            R_error[0, 2] - R_error[2, 0],
            R_error[1, 0] - R_error[0, 1]
        ]) / (2 * np.sin(angle))
    
    # 轴角表示：轴 * 角度
    rot_error = axis * angle
    
    # 组合位置和旋转误差
    return np.concatenate([pos_error, rot_error])

5. 自定义雅可比矩阵计算函数(有问题)

In [ ]:
def custom_jacobian(angles, dh_params):
    """
    关节角度的雅可比矩阵
    
    参数:
        angles: 当前关节角度 [theta1, theta2, theta3]
        dh_params: DH参数字典
        
    返回:
        6×3的雅可比矩阵，每列代表对应关节的影响
    """
    theta1, theta2, theta3 = angles
    # 提取DH参数
    a1 = dh_params.get('a1', 0)
    a2 = dh_params.get('a2', 0)
    d1 = dh_params.get('d1', 0)
    d2 = dh_params.get('d2', 0)
    d3 = dh_params.get('d3', 0)
    alpha1 = dh_params.get('alpha1', 0)
    alpha2 = dh_params.get('alpha2', 0)
    
    # 计算各个变换矩阵
    T0_1 = dh_transform(0, 0, d1, theta1)
    T1_2 = dh_transform(a1, alpha1, d2, theta2-np.pi/2)  # 第二个关节有90度的偏移
    T2_3 = dh_transform(a2, alpha2, d3, theta3)
    
    # 计算T0_2和T0_3
    T0_2 = np.dot(T0_1, T1_2)
    T0_3_current = np.dot(T0_2, T2_3)
    
    # 计算各关节坐标系的z轴方向（旋转轴）
    z0 = np.array([0, 0, 1])  # 基坐标系的z轴
    z1 = T0_1[:3, :3] @ np.array([0, 0, 1])  # 第1关节z轴在基坐标系中的表示
    z2 = T0_2[:3, :3] @ np.array([0, 0, 1])  # 第2关节z轴在基坐标系中的表示
    
    # 计算各关节的位置
    o0 = np.array([0, 0, 0])
    o1 = T0_1[:3, 3]
    o2 = T0_2[:3, 3]
    o3 = T0_3_current[:3, 3]  # 末端位置
    
    # 计算位置雅可比矩阵 (3×3)
    J_v = np.zeros((3, 3))
    
    # 关节1对末端位置的影响
    J_v[:, 0] = np.cross(z0, o3 - o0)
    
    # 关节2对末端位置的影响
    J_v[:, 1] = np.cross(z1, o3 - o1)
    
    # 关节3对末端位置的影响
    J_v[:, 2] = np.cross(z2, o3 - o2)
    
    # 计算旋转雅可比矩阵 (3×3)
    J_w = np.zeros((3, 3))
    J_w[:, 0] = z0  # 关节1的旋转轴
    J_w[:, 1] = z1  # 关节2的旋转轴
    J_w[:, 2] = z2  # 关节3的旋转轴
    
    # 合并为完整的雅可比矩阵 (6×3)
    J = np.vstack([J_v, J_w])
    
    return J

6. 数值雅可比矩阵计算函数

In [ ]:

def numerical_jacobian1(angles, dh_params, delta=1e-7):
    """
    使用数值微分法计算雅可比矩阵，避免使用欧拉角
    """
    theta1, theta2, theta3 = angles
    
    # 计算当前位姿矩阵
    T_current = forward_kinematics_first3joints(angles, dh_params)
    current_pos = T_current[:3, 3]
    current_rot = T_current[:3, :3]
    
    # 初始化雅可比矩阵
    J = np.zeros((6, 3))
    
    # 对每个关节计算雅可比列
    for i, (th1, th2, th3) in enumerate([
        (theta1 + delta, theta2, theta3),
        (theta1, theta2 + delta, theta3),
        (theta1, theta2, theta3 + delta)
    ]):
        # 计算扰动后的位姿
        T_plus = forward_kinematics_first3joints([th1, th2, th3], dh_params)
        pos_plus = T_plus[:3, 3]
        rot_plus = T_plus[:3, :3]
        
        # 计算位置雅可比列
        J[:3, i] = (pos_plus - current_pos) / delta
        
        # 计算旋转雅可比列 (使用对数映射)
        dR = rot_plus @ current_rot.T
        angle = np.arccos((np.trace(dR) - 1) / 2)
        if abs(angle) < 1e-10:
            w = np.zeros(3)
        else:
            w = np.array([
                dR[2, 1] - dR[1, 2],
                dR[0, 2] - dR[2, 0],
                dR[1, 0] - dR[0, 1]
            ]) * angle / (2 * np.sin(angle))
            
        J[3:, i] = w / delta
    
    return J

使用李代数和中心差分法计算雅可比矩阵

In [ ]:
def numerical_jacobian(angles, dh_params, delta=1e-6):
    """
    使用李代数和中心差分法计算雅可比矩阵
    """
    # 计算当前位姿矩阵
    T_current = forward_kinematics_first3joints(angles, dh_params)
    current_pos = T_current[:3, 3]
    current_rot = T_current[:3, :3]
    
    # 初始化雅可比矩阵
    J = np.zeros((6, 3))
    
    # 对每个关节计算雅可比列
    for i in range(3):
        # 创建前向和后向扰动的角度
        angles_plus = list(angles)
        angles_minus = list(angles)
        
        angles_plus[i] += delta
        angles_minus[i] -= delta
        
        # 计算前向扰动后的位姿
        T_plus = forward_kinematics_first3joints(angles_plus, dh_params)
        pos_plus = T_plus[:3, 3]
        rot_plus = T_plus[:3, :3]
        
        # 计算后向扰动后的位姿
        T_minus = forward_kinematics_first3joints(angles_minus, dh_params)
        pos_minus = T_minus[:3, 3]
        rot_minus = T_minus[:3, :3]
        
        # 计算位置雅可比列（中心差分）
        J[:3, i] = (pos_plus - pos_minus) / (2 * delta)
        
        # 使用李代数的对数映射计算旋转雅可比列
        log_plus = logSO3(rot_plus @ current_rot.T)
        log_minus = logSO3(rot_minus @ current_rot.T)
        
        # 使用中心差分
        J[3:, i] = (log_plus - log_minus) / (2 * delta)
    
    return J

def logSO3(R):
    """
    从旋转矩阵到so(3)的对数映射
    返回一个向量 w，使得 R = exp([w]_×)
    """
    # 确保R是一个有效的旋转矩阵
    if np.linalg.det(R) < 0:
        raise ValueError("不是有效的旋转矩阵（行列式为负）")
    
    # 计算旋转角度
    cos_theta = (np.trace(R) - 1) / 2
    cos_theta = np.clip(cos_theta, -1.0, 1.0)  # 确保在[-1,1]范围内
    theta = np.arccos(cos_theta)
    
    # 如果角度接近0，返回零向量
    if abs(theta) < 1e-10:
        return np.zeros(3)
    
    # 如果角度接近π
    if abs(theta - np.pi) < 1e-10:
        # 查找不为零的对角元
        if R[0, 0] > -1 + 1e-10:
            w = np.sqrt((1 + R[0, 0] - R[1, 1] - R[2, 2]) / 2)
            return np.array([w, R[0, 1]/(2*w), R[0, 2]/(2*w)]) * theta / np.sin(theta)
        elif R[1, 1] > -1 + 1e-10:
            w = np.sqrt((1 - R[0, 0] + R[1, 1] - R[2, 2]) / 2)
            return np.array([R[1, 0]/(2*w), w, R[1, 2]/(2*w)]) * theta / np.sin(theta)
        else:
            w = np.sqrt((1 - R[0, 0] - R[1, 1] + R[2, 2]) / 2)
            return np.array([R[2, 0]/(2*w), R[2, 1]/(2*w), w]) * theta / np.sin(theta)
    
    # 一般情况：提取旋转轴
    w = np.array([
        R[2, 1] - R[1, 2],
        R[0, 2] - R[2, 0],
        R[1, 0] - R[0, 1]
    ]) * theta / (2 * np.sin(theta))
    
    return w

7. Levenberg-Marquardt 算法实现

In [ ]:
class LMSolver:
    """
    完全按照KDL实现的Levenberg-Marquardt算法求解器
    """
    def __init__(self, dh_params,P,R):
        """
        初始化LM求解器
        
        参数:
            dh_params: DH参数字典
        """
        self.dh_params = dh_params
        self.nj = 3  # 我们求解前3个关节
        
        # 设置算法参数
        self.maxiter = 1000      # 最大迭代次数
        self.eps = 1e-15          # 位姿误差收敛阈值
        self.eps_joints = 1e-15   # 关节角度收敛阈值
        
        # 权重参数 - 可以调整位置和旋转误差的权重
        self.L = np.ones(6)
        self.L[:3] = P         # 位置误差权重
        self.L[3:] = R # 姿态误差权重
     
        # 结果信息
        self.lastNrOfIter = 0
        self.lastDifference = 0
        self.lastTransDiff = 0
        self.lastRotDiff = 0
        self.lastSV = None
        
        self.display_information = True  # 是否显示迭代信息
        
        self.enable_early_termination = True # 是否启用提前终止
    
    def cartToJnt(self, q_init, target_pose, jac_method='custom'):
        """
        使用LM算法求解逆运动学
        
        参数:
            q_init: 初始关节角度, 长度为nj的np.array
            target_pose: 目标位姿(4x4变换矩阵)
            jac_method: 雅可比矩阵计算方法 ('custom' 或 'numerical')
        
        返回:
            q_out: 求解得到的关节角度
            error: 错误码
            history: 迭代历史数据
        """
        # 检查参数尺寸
        if len(q_init) != self.nj:
            return q_init, self.E_SIZE_MISMATCH, None
        
        # 初始化历史记录
        history = {
            'iterations': [],
            'q': [q_init.copy()],
            'errors': [],
            'rho':[]
        }
        
        # 初始化基本参数 
        v = 2         # 步长因子
        tau = 1e-3  # 初始阻尼因子系数
        
        # 复制初始关节角度
        q = q_init.copy()
        q_out = q.copy()  # 初始化输出变量
        
        # 计算初始位姿 - 对应compute_fwdpos(q)
        T_base_head = forward_kinematics_first3joints(q, self.dh_params)
        
        # 计算初始误差 - 对应Twist_to_Eigen(diff(...))
        delta_pos = compute_pose_error(T_base_head, target_pose)
        
        # 应用权重 - 对应L.asDiagonal()*delta_pos
        delta_pos = self.L * delta_pos
        
        # 计算误差范数 - 对应delta_pos.norm()
        delta_pos_norm = np.linalg.norm(delta_pos)
        
        # 检查初始误差是否已经足够小
        if delta_pos_norm < self.eps:
            self.lastNrOfIter = 0
            
            # 重新计算未加权的误差 - 对应Twist_to_Eigen(diff(...))
            delta_pos_unweighted = compute_pose_error(T_base_head, target_pose)
            self.lastDifference = np.linalg.norm(delta_pos_unweighted)
            self.lastTransDiff = np.linalg.norm(delta_pos_unweighted[:3])
            self.lastRotDiff = np.linalg.norm(delta_pos_unweighted[3:])
            
            # 计算雅可比矩阵
            if jac_method == 'custom':
                jac = custom_jacobian(q, self.dh_params)
            else:  # numerical
                jac = numerical_jacobian(q, self.dh_params)
            
            jac_weighted = np.diag(self.L) @ jac
            
            # 计算SVD
            _, s, _ = np.linalg.svd(jac_weighted, full_matrices=False)
            self.lastSV = s
            
            return q, self.E_NOERROR, history
        
        # 计算雅可比矩阵 - 对应compute_jacobian(q)
        if jac_method == 'custom':
            jac = custom_jacobian(q, self.dh_params)
        else:  # numerical
            jac = numerical_jacobian(q, self.dh_params)
        
        # 应用权重到雅可比矩阵 - 对应jac = L.asDiagonal()*jac
        jac = np.diag(self.L) @ jac
        
        # 初始化lambda (按照论文实现)
        lambda_ = tau * np.max(jac.T @ jac)
        
        # 主迭代循环
        for i in range(self.maxiter):
            # 计算SVD (每次迭代都重新计算) - 对应svd.compute(jac)
            U, s, Vh = np.linalg.svd(jac, full_matrices=False)

            # 修改奇异值 - 对应修改original_Aii
            original_Aii = np.zeros_like(s)
            for j in range(len(s)):  # 对所有奇异值应用
                original_Aii[j] = s[j] / (s[j]*s[j] + lambda_)
            
            # 计算关节角增量 - 对应KDL中的Segment A部分
            tmp = U.T @ delta_pos  
            tmp = tmp * original_Aii
            diffq = Vh.T @ tmp

             # 计算梯度 - 对应grad = jac.transpose()*delta_pos
            grad = jac.T @ delta_pos

            # 修改为： (jac.T @ jac + lambda_ * np.eye(self.nj)) @ diffq = grad
            #A = jac.T @ jac + lambda_ * np.eye(self.nj)
            #diffq = np.linalg.inv(A) @ grad
            
            # 输出迭代信息
            if self.display_information and (i % 100 == 0 or i < 5):
                print(f"------- iteration {i} ----------------")
                print(f"  q              = {q}")
                print(f"  lambda         = {lambda_}")
                print(f"  eigenvalues    = {s}")
                print(f"  difference norm= {delta_pos_norm}")
                print(f"  grad norm      = {np.linalg.norm(grad)}")
                print("")
            
            # 记录历史
            history['iterations'].append(i)
            history['q'].append(q.copy())
            history['errors'].append(delta_pos_norm)
            
            # 检查步长是否足够小 - 对应dnorm = diffq.lpNorm<Eigen::Infinity>()
            dnorm = np.max(np.abs(diffq))  # 无穷范数
            if self.enable_early_termination and dnorm < self.eps_joints:
                self.lastDifference = delta_pos_norm
                self.lastNrOfIter = i
                self.lastSV = s
                q_out = q.copy()
                
                # 重新计算当前位姿和误差
                T_base_head = forward_kinematics_first3joints(q, self.dh_params)
                delta_pos_unweighted = compute_pose_error(T_base_head, target_pose)
                self.lastTransDiff = np.linalg.norm(delta_pos_unweighted[:3])
                self.lastRotDiff = np.linalg.norm(delta_pos_unweighted[3:])
                print(f"步长过小，停止迭代,迭代次数：{i}")
                return q_out, history
            

            # 检查梯度是否足够小 - 对应grad.transpose()*grad < eps_joints*eps_joints
            if self.enable_early_termination and np.dot(grad, grad) < self.eps_joints * self.eps_joints:
                # 重新计算当前位姿和误差
                T_base_head = forward_kinematics_first3joints(q, self.dh_params)
                delta_pos_unweighted = compute_pose_error(T_base_head, target_pose)
                
                self.lastDifference = np.linalg.norm(delta_pos_unweighted)
                self.lastTransDiff = np.linalg.norm(delta_pos_unweighted[:3])
                self.lastRotDiff = np.linalg.norm(delta_pos_unweighted[3:])
                self.lastSV = s
                self.lastNrOfIter = i
                q_out = q.copy()
                print(f"梯度过小，停止迭代,迭代次数：{i}")
                return q_out, history
            
            # 计算新的关节角度 - 对应q_new = q+diffq
            q_new = q +  diffq
            #q_new = np.clip(q_new, [-170*np.pi/180,-105*np.pi/180,-210*np.pi/180],[170*np.pi/180,145*np.pi/180,70*np.pi/180] )
            # 计算新的位姿和误差
            T_base_head_new = forward_kinematics_first3joints(q_new, self.dh_params)
            delta_pos_new = compute_pose_error(T_base_head_new, target_pose)
            delta_pos_new = self.L * delta_pos_new  # 应用权重
            delta_pos_new_norm = np.linalg.norm(delta_pos_new)
            
            # 计算信任区域比率 - 对应KDL中的rho计算
            rho = delta_pos_norm**2 - delta_pos_new_norm**2
            history['rho'].append(rho)
            denominator = diffq.T @ (lambda_ * diffq + grad)
            
            # 防止除零
            if abs(denominator) < 1e-10:
                if rho > 0:
                    rho = 1.0
                else:
                    rho = -1.0
            else:
                rho = rho / denominator
            
            # 根据信任区域比率更新参数
            if rho > 0:
                # 更新成功，接受新的关节角度
                q = q_new.copy()
                delta_pos = delta_pos_new.copy()
                delta_pos_norm = delta_pos_new_norm
                grad = jac.T @ delta_pos
                # 检查是否收敛 按照文献流程修改
                if self.enable_early_termination and (np.max(np.abs(grad))<self.eps or delta_pos_norm* delta_pos_norm < self.eps):
                    # 重新计算未加权的误差
                    delta_pos_unweighted = compute_pose_error(T_base_head_new, target_pose)
                    self.lastDifference = np.linalg.norm(delta_pos_unweighted)
                    self.lastTransDiff = np.linalg.norm(delta_pos_unweighted[:3])
                    self.lastRotDiff = np.linalg.norm(delta_pos_unweighted[3:])
                    self.lastSV = s
                    self.lastNrOfIter = i
                    q_out = q.copy()
                    print(f"收敛，停止迭代,迭代次数：{i}")
                    return q_out, self.E_NOERROR, history
                
                # 重新计算雅可比矩阵 - 使用新的关节角度
                if jac_method == 'custom':
                    jac = custom_jacobian(q, self.dh_params)
                else:  # numerical
                    jac = numerical_jacobian(q, self.dh_params)
                
                jac = np.diag(self.L) @ jac  # 应用权重
               
                # 减小阻尼因子 (KDL方式) - 对应lambda = lambda*max(1/3.0, 1-tmp*tmp*tmp)
                fmp = 2 * rho - 1
                lambda_ = lambda_ * max(1/3, 1 - fmp**3)
                v = 2
            else:
                # 更新失败，增大阻尼因子 - 对应lambda = lambda*v; v = 2*v
                lambda_ = lambda_ * v
                v = 2 * v
        
        # 达到最大迭代次数
        self.lastDifference = delta_pos_norm
        self.lastTransDiff = np.linalg.norm(delta_pos[:3] / self.L[:3])
        self.lastRotDiff = np.linalg.norm(delta_pos[3:] / self.L[3:])
        self.lastSV = s
        self.lastNrOfIter = self.maxiter
        q_out = q.copy()
        
        print(f"达到最大迭代次数 {self.maxiter}")
        print(f"最终位置误差: {self.lastTransDiff}")
        print(f"最终旋转误差: {self.lastRotDiff}")
        
        return q_out, history

8. 实际T03计算函数

In [ ]:
def calculate_wrist_point(tcp_pose, theta4, theta5, theta6, dh_params):
    """
    
    参数:
        tcp_pose: TCP位姿矩阵 (4x4)
        theta4, theta5, theta6: 腕关节角度 (弧度)
        dh_params: DH参数字典
        
    返回:
        wrist_pose: T03位姿矩阵 (4x4)
    """
    # 提取相关DH参数
    a3 = dh_params.get('a3', 0)
    a4 = dh_params.get('a4', 0)
    a5 = dh_params.get('a5', 0)
    
    d4 = dh_params.get('d4', 0)
    d5 = dh_params.get('d5', 0)
    d6 = dh_params.get('d6', 0)
    
    alpha3 = dh_params.get('alpha3', 0)
    alpha4 = dh_params.get('alpha4', 0)
    alpha5 = dh_params.get('alpha5', 0)
    
    # 计算腕部变换矩阵
    T3_4 = dh_transform(a3, alpha3, d4, theta4)
    T4_5 = dh_transform(a4, alpha4, d5, theta5)
    T5_6 = dh_transform(a5, alpha5, d6, theta6 + np.pi)  # 腕部末端偏移180度
    
    # 计算腕部到TCP的变换矩阵
    T3_6 = np.dot(T3_4, np.dot(T4_5, T5_6))
  
    # 计算腕点位姿
    T0_3 = np.dot(tcp_pose, np.linalg.inv(T3_6))
    
    return T0_3

9. 完整的逆运动学求解函数

In [ ]:
def solve_inverse_kinematics_lm(x, y, z, nx, ny, nz, ox, oy, oz, ax, ay, az, 
                              theta4, theta5, theta6, dh_params, p=1,r=100,
                              initial_guess=None, jac_method='custom'):
    """
    使用LM算法求解6轴机器人逆运动学前3个关节角度
    
    参数:
        x, y, z: TCP位置坐标
        nx, ny, nz: TCP位姿的x轴方向
        ox, oy, oz: TCP位姿的y轴方向
        ax, ay, az: TCP位姿的z轴方向
        theta4, theta5, theta6: 已知的关节角度(弧度)
        dh_params: 机器人DH参数字典
        initial_guess: 初始关节角度猜测值，默认为零位
        jac_method: 雅可比矩阵计算方法 ('custom' 或 'numerical')
        
    返回:
        theta1, theta2, theta3: 计算得到的前3个关节角度(弧度)
        error_code: 错误码
    """
    # 构建TCP位姿矩阵
    tcp_pose = np.array([
        [nx, ox, ax, x],
        [ny, oy, ay, y],
        [nz, oz, az, z],
        [0, 0, 0, 1]
    ])
    
    # 计算腕点位置 (从TCP向后推算)T03矩阵
    T03 = calculate_wrist_point(tcp_pose, theta4, theta5, theta6, dh_params)
    
    # 初始化LM求解器
    solver = LMSolver(dh_params,p,r)
    solver.display_information = False  # 是否显示迭代信息
    
    # 设置初始猜测值
    if initial_guess is None:
        initial_guess = np.zeros(3)  # 默认全部为0
    
    # 使用LM算法求解前3个关节角度
    q_sol,history= solver.cartToJnt(initial_guess, T03, jac_method)
    
    # 返回计算得到的关节角度
    theta1, theta2, theta3 = q_sol
    
    # 规范化角度到[-pi, pi]范围域
    theta1 = ((theta1 + np.pi) % (2*np.pi)) - np.pi
    theta2 = ((theta2 + np.pi) % (2*np.pi)) - np.pi
    theta3 = ((theta3 + np.pi) % (2*np.pi)) - np.pi
    
    return theta1, theta2, theta3,history

In [ ]:
def solve_with_multiple_initial_guesses(x, y, z, nx, ny, nz, ox, oy, oz, ax, ay, az, 
                                       theta4, theta5, theta6, dh_params, jac_method='custom'):
    """
    使用多个初始猜测值尝试求解逆运动学
    """
    initial_guesses = [
        [0.0, 0.0, 0.0],
        [np.pi/4, np.pi/4, np.pi/4],
        [-np.pi/4, np.pi/4, np.pi/4],
        [np.pi/4, -np.pi/4, np.pi/4],
        [np.pi/4, np.pi/4, -np.pi/4],
        [np.pi/2, 0.0, 0.0],
        [0.0, np.pi/2, 0.0],
        [0.0, 0.0, np.pi/2],
        [np.pi/3, np.pi/3, np.pi/3],
        [-np.pi/3, -np.pi/3, -np.pi/3],
        [np.pi/6, np.pi/6, np.pi/6],
        [-np.pi/6, -np.pi/6, -np.pi/6]
    ]
    
    # 跟踪最佳结果
    best_result = None
    best_error = float('inf')

    
    # 尝试每个初始猜测值
    for i, guess in enumerate(initial_guesses):
        
        try:
            # 求解逆运动学
            theta1, theta2, theta3,history= solve_inverse_kinematics_lm(
                x, y, z, nx, ny, nz, ox, oy, oz, ax, ay, az, 
                theta4, theta5, theta6, dh_params, p=1,r=10000,
                initial_guess=guess,
                jac_method=jac_method
            )
            
            # 验证结果
            joint_angles = [theta1, theta2, theta3, theta4, theta5, theta6]
            result_pose = forward_kinematics(joint_angles, dh_params)
            result_pos = result_pose[:3, 3]
            
            # 计算误差
            position_error = np.linalg.norm(result_pos - np.array([x, y, z]))
            
            # 检查是否是最佳结果
            if position_error < best_error:
                best_error = position_error
                best_result = (theta1, theta2, theta3)
           
        except Exception as e:
            print(f"  求解出错: {e}")
            continue
    
    
    # 输出最佳结果
    if best_result is not None:
        theta1, theta2, theta3 = best_result
        print(f"\n===== 最佳解 =====")
        print(f"关节角度: θ1={np.degrees(theta1):.1f}°, θ2={np.degrees(theta2):.1f}°, θ3={np.degrees(theta3):.1f}°")
        print(f"位置误差: {best_error:.4f} mm")
        
        return theta1, theta2, theta3 ,history
    else:
        print("\n未找到有效解!")
        return None, None, None

插补计算

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R
def slerp(q1, q2, t):
    """
    实现四元数球面线性插值
    
    参数:
        q1: 起始四元数 [x, y, z, w]
        q2: 终止四元数 [x, y, z, w]
        t: 插值参数 [0, 1]
    """
    # 确保四元数为单位四元数
    q1 = q1 / np.linalg.norm(q1)
    q2 = q2 / np.linalg.norm(q2)
    
    # 计算四元数点积
    dot = np.dot(q1, q2)
    
    # 如果点积为负，翻转其中一个四元数
    if dot < 0.0:
        q2 = -q2
        dot = -dot
    
    # 如果四元数非常接近，使用线性插值
    if dot > 0.9995:
        result = q1 + t * (q2 - q1)
        return result / np.linalg.norm(result)
    
    # 执行球面线性插值
    theta_0 = np.arccos(dot)
    sin_theta_0 = np.sin(theta_0)
    
    theta = theta_0 * t
    sin_theta = np.sin(theta)
    
    s0 = np.cos(theta) - dot * sin_theta / sin_theta_0
    s1 = sin_theta / sin_theta_0
    
    return s0 * q1 + s1 * q2

def cartesian_linear_interpolation(start_position, end_position, start_quat, end_quat, num_points):
    """
    笛卡尔空间线性插补
    
    参数:
        start_position: 起始TCP位置 [x, y, z]
        end_position: 终止TCP位置 [x, y, z]
        start_quat: 起始四元数 [x, y, z, w]
        end_quat: 终止四元数 [x, y, z, w]
        num_points: 插补点数量
        
    返回:
        positions: 插补后的位置列表
        quats: 插补后的四元数列表
    """
    positions = []
    quats = []
    
    # 确保四元数格式正确
    start_rot = start_quat
    end_rot = end_quat
    
    for i in range(num_points):
        t = i / (num_points - 1)
        
        # 线性插值位置
        pos = (1 - t) * np.array(start_position) + t * np.array(end_position)
        positions.append(pos)
        
        # 球面线性插值四元数
        interpolated_rot = slerp(start_rot, end_rot, t)
        quats.append(interpolated_rot)
    
    return positions, quats

def joint_linear_interpolation(start_joints, end_joints, num_points):
    """
    关节空间线性插补
    
    参数:
        start_joints: 起始关节角度 [j4, j5, j6]
        end_joints: 终止关节角度 [j4, j5, j6]
        num_points: 插补点数量
        
    返回:
        joints: 插补后的关节角度列表
    """
    joints = []
    start_joints = np.array(start_joints)
    end_joints = np.array(end_joints)
    
    for i in range(num_points):
        t = i / (num_points - 1)
        # 线性插值关节角度
        joint = (1 - t) * start_joints + t * end_joints
        joints.append(joint)
    
    return joints

def get_wrist_center(position, orientation, d6):
    """
    计算腕中心位置
    
    参数:
        position: TCP位置 [x, y, z]
        orientation: 四元数 [x, y, z, w]
        d6: 腕关节到TCP的偏移距离
        
    返回:
        wrist_center: 腕中心位置 [x, y, z]
    """
    # 将四元数转换为旋转矩阵
    rot = R.from_quat(orientation).as_matrix()
    
    # 计算工具z轴方向
    z_axis = rot[:, 2]
    
    # 腕中心位置 = TCP位置 - d6 * 工具z轴方向
    wrist_center = np.array(position) - d6 * z_axis
    
    return wrist_center

def interpolation_for_integration(start_position, end_position, start_quat, end_quat, 
                                  start_wrist_joints, end_wrist_joints, d6, num_points=50):
    """
    生成用于集成到您系统的插补数据
    
    参数:
        start_position: 起始TCP位置 [x, y, z]
        end_position: 终止TCP位置 [x, y, z]
        start_quat: 起始四元数 [x, y, z, w]
        end_quat: 终止四元数 [x, y, z, w]
        start_wrist_joints: 起始腕关节角度 [j4, j5, j6]
        end_wrist_joints: 终止腕关节角度 [j4, j5, j6]
        d6: 工具长度参数
        num_points: 插补点数量
        
    返回:
        插补结果字典
    """
    # 笛卡尔空间插补
    positions, quats = cartesian_linear_interpolation(
        start_position, end_position, start_quat, end_quat, num_points)
    
    # 关节空间插补
    wrist_joints = joint_linear_interpolation(
        start_wrist_joints, end_wrist_joints, num_points)
    
    # 计算腕中心位置
    wrist_centers = []
    for i in range(num_points):
        wc = get_wrist_center(positions[i], quats[i], d6)
        wrist_centers.append(wc)
    
    # 返回所有需要的数据
    return {
        'positions': np.array(positions),
        'quats': np.array(quats),
        'wrist_joints': np.array(wrist_joints),
        'wrist_centers': np.array(wrist_centers)
    }

单点求解

In [ ]:

from scipy.spatial.transform import Rotation 

x, y, z = 1078.478,268.266,565.881  # 位置(mm)  
quat = [-0.5804,0.7087,-0.3541,-0.1885] #[x y z w]  # 四元数
# 已知的关节4,5,6角度(弧度)  
theta4 = np.radians(-51.241)  
theta5 = np.radians(69.480)  
theta6 = np.radians(-34.404) 
# 创建旋转对象
rot = Rotation.from_quat(quat)

# 获取旋转矩阵
rotation_matrix = rot.as_matrix() 

# 提取姿态矩阵的各列向量  
nx,ny,nz = rotation_matrix[:,0]  # x轴方向向量
ox,oy,oz = rotation_matrix[:,1]  # y轴方向向量  
ax,ay,az = rotation_matrix[:,2]  # z轴方向向量

# 设置机器人DH参数(工业机器人 NB12s-1214-5A)  
dh_params = {  
    'a0': 0,      # 连杆1长度(mm)
    'a1': 100,    # 连杆2长度(mm) 
    'a2': 680,    # 连杆3长度(mm)
    'a3': 50,     # 连杆4长度(mm)
    'a4': 0,      # 连杆5长度(mm)
    'a5': 0,      # 连杆6长度(mm)
    'd1': 500,    # 关节1偏移(mm)
    'd2': 0,      # 关节2偏移(mm)
    'd3': 0,      # 关节3偏移(mm
    'd4': 660,    # 关节4偏移(mm)
    'd5': 0,      # 关节5偏移(mm)
    'd6': 98,     # 关节6偏移(mm)
    'alpha0': 0,          # 连杆0扭角(弧度)
    'alpha1': -np.pi/2,   # 连杆1扭角(弧度)
    'alpha2': 0,          # 连杆2扭角(弧度)
    'alpha3': -np.pi/2,   # 连杆3扭角(弧度)
    'alpha4': np.pi/2,    # 连杆4扭角(弧度)
    'alpha5': -np.pi/2,   # 连杆5扭角(弧度)
}

print("===== 使用LM算法求解6轴机器人逆运动学 =====")
print(f"目标TCP位置: [{x}, {y}, {z}] mm")
print(f"目标TCP姿态: nx={nx:.3f}, ny={ny:.3f}, nz={nz:.3f}, ox={ox:.3f}, oy={oy:.3f}, oz={oz:.3f}, ax={ax:.3f}, ay={ay:.3f}, az={az:.3f}")

# 构建TCP位姿矩阵
tcp_pose = np.array([
    [nx, ox, ax, x],
    [ny, oy, ay, y],
    [nz, oz, az, z],
    [0, 0, 0, 1]
])

print("TCP位姿矩阵:")
print(tcp_pose)
# 开始计时
start_time = time.time()

# 使用自定义雅可比矩阵的LM算法求解
print("\n使用自定义雅可比矩阵的LM算法求解...")
theta1, theta2, theta3 ,history= solve_with_multiple_initial_guesses(
    x, y, z, nx, ny, nz, ox, oy, oz, ax, ay, az, 
    theta4, theta5, theta6, dh_params, 
    jac_method='nuemnails'
)

# 计算求解时间
elapsed_time = time.time() - start_time
print(f"求解时间: {elapsed_time:.4f} 秒")

#S实际T03矩阵
T03_pre = calculate_wrist_point(tcp_pose, theta4, theta5, theta6, dh_params)
print("\n实际T03矩阵:")
print(T03_pre)
#计算T0300
T03= forward_kinematics_first3joints([theta1, theta2, theta3], dh_params)
print("\n计算T03矩阵:")
print(T03)

# 验证求解结果
print("\n验证求解结果:")

# 所有关节角度
joint_angles = [theta1, theta2, theta3, theta4, theta5, theta6]

# 使用正向运动学验证
result_pose = forward_kinematics(joint_angles, dh_params)
result_pos = result_pose[:3, 3]

# 计算位置误差
position_error = np.linalg.norm(result_pos - np.array([x, y, z]))

print(f"原始TCP位置: [{x}, {y}, {z}] mm")
print(f"验证TCP位置: [{result_pos[0]:.3f}, {result_pos[1]:.3f}, {result_pos[2]:.3f}] mm")
print(f"位置误差: {position_error:.6f} mm")


# 比较姿态矩阵
result_rot = result_pose[:3, :3]
rotation_error = np.linalg.norm(rotation_matrix - result_rot)
print(f"姿态误差: {rotation_error:.6f}")

# 打印所有关节角度
print("\n完整关节角度:")
print(f"关节1: {np.degrees(theta1):.3f}°")
print(f"关节2: {np.degrees(theta2):.3f}°")
print(f"关节3: {np.degrees(theta3):.3f}°")
print(f"关节4: {np.degrees(theta4):.3f}°")
print(f"关节5: {np.degrees(theta5):.3f}°")
print(f"关节6: {np.degrees(theta6):.3f}°")


## 轨迹求解

In [ ]:
def test_multiple_points(points_data):
    """测试多个路点的逆运动学求解
    
    参数:
        points_data: 包含多个路点信息的列表,每个路点是一个字典,包含:
            - position: [x, y, z]
            - quaternion: [qx, qy, qz, qw] 
            - wrist_joints: [theta4, theta5, theta6](degrees)
    """
    results = []
    poss = points_data['positions']
    quats = points_data['quats']
    wrists = [np.radians(a) for a in points_data['wrist_joints']]

    for i in range(50):
        print(f"\n========= 路点 {i+1} =========")
        pos = poss[i]
        quat = quats[i]
        wrist = wrists[i]
        print(f"位置: [{pos[0]:.3f}, {pos[1]:.3f}, {pos[2]:.3f}] mm")
        print(f"四元数: [{quat[0]:.4f}, {quat[1]:.4f}, {quat[2]:.4f}, {quat[3]:.4f}]")
        print(f"后三个关节角度: [{wrist[0]:.2f}, {wrist[1]:.2f}, {wrist[2]:.2f}] rad")
        
        # 创建旋转对象并获取旋转矩阵
        rot = Rotation.from_quat(quat)
        rotation_matrix = rot.as_matrix()
        
        # 提取姿态向量
        nx,ny,nz = rotation_matrix[:,0]
        ox,oy,oz = rotation_matrix[:,1]
        ax,ay,az = rotation_matrix[:,2]
        
        if len(results) == 0:
            initial_guess = [0.0, 0.0, 0.0]  # 默认初始猜测值
        else:
            initial_guess = [np.radians(a) for a in results[-1]['joint_angles'][:3]]
        print(f"初始值：{initial_guess[0]:.2f},{initial_guess[1]:.2f},{initial_guess[2]:.2f}rad")   
        # 
        theta1_c, theta2_c, theta3_c, history_c = solve_inverse_kinematics_lm(
            pos[0], pos[1], pos[2], 
            nx, ny, nz, ox, oy, oz, ax, ay, az,
            wrist[0], wrist[1], wrist[2], 
            dh_params,p=1,r=100000,initial_guess= initial_guess,
            jac_method='numerical'
        )
        
        # 验证结果
        joint_angles = [theta1_c, theta2_c, theta3_c, wrist[0], wrist[1], wrist[2]]
        result_pose = forward_kinematics(joint_angles, dh_params)
        
        error =  result_pose[:3, 3] - np.array([pos[0], pos[1], pos[2]])
        error = np.linalg.norm(error)

        #求解T46矩阵
           # 提取DH参数
        a3 = dh_params.get('a3', 0)
        a4 = dh_params.get('a4', 0)
        a5 = dh_params.get('a5', 0)
        
        d4 = dh_params.get('d4', 0)
        d5 = dh_params.get('d5', 0)
        d6 = dh_params.get('d6', 0)
        alpha3 = dh_params.get('alpha3', 0)
        alpha4 = dh_params.get('alpha4', 0)
        alpha5 = dh_params.get('alpha5', 0)

        T3_4 = dh_transform(a3, alpha3, d4, wrist[0])
        T4_5 = dh_transform(a4, alpha4, d5, wrist[1])
        T5_6 = dh_transform(a5, alpha5, d6, wrist[2]+np.pi) #  第六个关节偏移180度
        T3_6 = np.dot(T3_4, np.dot(T4_5, T5_6))

        results.append({
            'joint_angles': [np.degrees(a) for a in joint_angles],
            'position_error': error,
            'history': history_c
        })
        
        # 打印该路点的结果
        print("\n求解结果:")
        print(f"位置误差: {error:.4f} mm")
        print("关节角度(度):")
        for j, angle in enumerate(results[-1]['joint_angles']):
            print(f"J{j+1}: {angle:.3f}°")
            
    return results

# 定义测试路点
test_points = [
    {
        'position': [913.612, 745.315, 633.829],
        'quaternion': [0.8465, -0.3332, 0.3434, 0.2335],
        'wrist_angles': [-50.023, 77.220, -75.153]
    },
    {
        'position': [1078.478, 268.266, 565.881],
        'quaternion': [-0.5804, 0.7087, -0.3541, -0.1885],
        'wrist_angles': [-51.241, 69.480, -34.404]
    },
    {
        'position': [915.249, 608.436, 233.411],
        'quaternion': [0.3604, 0.8224, 0.3919, -0.2006],
        'wrist_angles': [90, 45, 0]
    }
]


start_pos=[1018.1418,40.4859,656.71229] # 起始位置
end_pos=[1018.1418,40.4859,1015.41402] # 目标位置
start_quat=[-0.0163,0.8205,0.0114,0.5713] # 起始四元数
end_quat=[-0.0163,0.8205,0.0114,0.5713] # 目标四元数
start_joint=[-0.00,-22.770,0.00]  # 起始关节角
end_joint=[-180,-6.578,180] # 目标关节角

points=interpolation_for_integration(start_pos,end_pos,start_quat,end_quat,start_joint,end_joint,d6=15)
# 运行测试
results = test_multiple_points(points)

# 统计分析
print("\n=== 统计分析 ===")
errors = [r['position_error'] for r in results]
print(f"平均位置误差: {np.mean(errors):.4f} mm")
print(f"最大位置误差: {np.max(errors):.4f} mm")
print(f"最小位置误差: {np.min(errors):.4f} mm")
print(f"误差标准差: {np.std(errors):.4f} mm")

绘图

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

# 绘制关节角度曲线
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12))

# 提取所有关节角度数据
joint_angles = np.array([result['joint_angles'] for result in results])

# 绘制前3个关节角度（算法求解得到的）
time_points = np.arange(len(joint_angles))
ax1.plot(time_points, joint_angles[:, 0], 'r-', label='J1')
ax1.plot(time_points, joint_angles[:, 1], 'g-', label='J2')
ax1.plot(time_points, joint_angles[:, 2], 'b-', label='J3')
ax1.set_title('前3个关节角度变化曲线')
ax1.set_xlabel('插补点序号')
ax1.set_ylabel('关节角度 (度)')
ax1.grid(True)
ax1.legend()

# 绘制后3个关节角度（线性插值得到的）
ax2.plot(time_points, joint_angles[:, 3], 'r--', label='J4')
ax2.plot(time_points, joint_angles[:, 4], 'g--', label='J5')
ax2.plot(time_points, joint_angles[:, 5], 'b--', label='J6')
ax2.set_title('后3个关节角度变化曲线')
ax2.set_xlabel('插补点序号')
ax2.set_ylabel('关节角度 (度)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()

# 绘制位置误差随时间的变化
plt.figure(figsize=(10, 6))
plt.plot(time_points, [r['position_error'] for r in results], 'k-', label='位置误差')
plt.title('插补过程中的位置误差变化')
plt.xlabel('插补点序号')
plt.ylabel('位置误差 (mm)')
plt.grid(True)
plt.legend()
plt.show()